![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/openvino/HuggingFace_OpenVINO_in_Spark_NLP_AlbertForTokenClassification.ipynb)

# Import OpenVINO AlbertForTokenClassification   models from HuggingFace 🤗 into Spark NLP 🚀

This notebook provides a detailed walkthrough on optimizing and exporting AlbertForTokenClassification   models from HuggingFace for use in Spark NLP, leveraging the various tools provided in the [Intel OpenVINO toolkit](https://www.intel.com/content/www/us/en/developer/tools/openvino-toolkit/overview.html) ecosystem.

Let's keep in mind a few things before we start 😊

- OpenVINO support was introduced in  `Spark NLP 5.4.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for AlbertForTokenClassification   from AlbertForTokenClassification   and they have to be in `Token Classification` category.

## 1. Export and Save the HuggingFace model

- Let's install `transformers` and `openvino` packages with other dependencies. You don't need `openvino` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [ ]:
!pip install -q --upgrade transformers==4.48.3 optimum[openvino]

[Optimum Intel](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#openvino) is the interface between the Transformers library and the various model optimization and acceleration tools provided by Intel. HuggingFace models loaded with optimum-intel are automatically optimized for OpenVINO, while being compatible with the Transformers API.
- To load a HuggingFace model directly for inference/export, just replace the `AutoModelForXxx` class with the corresponding `OVModelForXxx` class. We can use this to import and export OpenVINO models with `from_pretrained` and `save_pretrained`.
- By setting `export=True`, the source model is converted to OpenVINO IR format on the fly.
- We'll use [HooshvareLab/albert-fa-zwnj-base-v2-ner](https://huggingface.co/HooshvareLab/albert-fa-zwnj-base-v2-ner) model from HuggingFace as an example and load it as a `OVModelForTokenClassification`, representing an OpenVINO model.
- In addition to the AlbertForTokenClassification   model, we also need to save the `AlbertTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [ ]:
from transformers import AutoTokenizer
from optimum.intel import OVModelForTokenClassification

MODEL_NAME = "HooshvareLab/albert-fa-zwnj-base-v2-ner"
EXPORT_PATH = f"ov_models/{MODEL_NAME}"

ort_model = OVModelForTokenClassification.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(EXPORT_PATH)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(EXPORT_PATH)

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/44.5M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/499 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/857k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.23M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/186 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/44.4M [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


('onnx_models/HooshvareLab/albert-fa-zwnj-base-v2-ner/tokenizer_config.json',
 'onnx_models/HooshvareLab/albert-fa-zwnj-base-v2-ner/special_tokens_map.json',
 'onnx_models/HooshvareLab/albert-fa-zwnj-base-v2-ner/spiece.model',
 'onnx_models/HooshvareLab/albert-fa-zwnj-base-v2-ner/added_tokens.json',
 'onnx_models/HooshvareLab/albert-fa-zwnj-base-v2-ner/tokenizer.json')

Let's have a look inside these two directories and see what we are dealing with:

In [4]:
!ls -l {EXPORT_PATH}

total 46948
-rw-r--r-- 1 root root     1630 Jun 18 01:15 config.json
-rw-r--r-- 1 root root 44440820 Jun 18 01:15 openvino_model.bin
-rw-r--r-- 1 root root   357319 Jun 18 01:15 openvino_model.xml
-rw-r--r-- 1 root root      971 Jun 18 01:15 special_tokens_map.json
-rw-r--r-- 1 root root   857476 Jun 18 01:15 spiece.model
-rw-r--r-- 1 root root    19227 Jun 18 01:15 tokenizer_config.json
-rw-r--r-- 1 root root  2381031 Jun 18 01:15 tokenizer.json


- We need to move the `spiece.model` file from the tokenizer into an assets folder, as this is where Spark NLP looks for it when working with models like Albert or other SentencePiece-based tokenizers.
- Additionally, we need to extract the `labels` and their corresponding `ids` from the model's config. This mapping will be saved as `labels.txt` inside the same `assets` folder.

In [5]:
!mkdir {EXPORT_PATH}/assets

labels = ort_model.config.label2id
labels = sorted(labels, key=labels.get)

with open(EXPORT_PATH + '/assets/labels.txt', 'w') as f:
    f.write('\n'.join(labels))

!mv {EXPORT_PATH}/spiece.model {EXPORT_PATH}/assets

Let's have a look inside these two directories and see what we are dealing with:

In [6]:
!ls -l {EXPORT_PATH}/assets

total 844
-rw-r--r-- 1 root root    121 Jun 18 01:15 labels.txt
-rw-r--r-- 1 root root 857476 Jun 18 01:15 spiece.model


Voila! We have our `spiece.model` and `labels.txt` inside assets directory

## Import and Save AlbertForTokenClassification in Spark NLP


- Install and set up Spark NLP in Google Colab
- This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [7]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 36.0 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [8]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `AlbertForTokenClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `AlbertForTokenClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively



In [9]:
from sparknlp.annotator import AlbertForTokenClassification

tokenClassifier = AlbertForTokenClassification\
  .loadSavedModel(EXPORT_PATH, spark)\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")\
  .setCaseSensitive(False)\
  .setMaxSentenceLength(128)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [10]:
tokenClassifier.write().overwrite().save("./{}_spark_nlp_openvino".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [11]:
!rm -rf {EXPORT_PATH}

Awesome 😎  !

This is your AlbertForTokenClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [12]:
! ls -l {MODEL_NAME}_spark_nlp_openvino

total 44604
-rw-r--r-- 1 root root 44805257 Jun 18 01:17 albert_classification_openvino
-rw-r--r-- 1 root root   857476 Jun 18 01:17 albert_spp
drwxr-xr-x 3 root root     4096 Jun 18 01:17 fields
drwxr-xr-x 2 root root     4096 Jun 18 01:17 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny AlbertForTokenClassification model 😊

In [13]:
tokenClassifier_loaded = AlbertForTokenClassification.load("./{}_spark_nlp_openvino".format(MODEL_NAME))\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")

You can see what labels were used to train this model via `getClasses` function:

In [14]:
tokenClassifier_loaded.getClasses()

['I-PCT',
 'B-PRO',
 'I-EVE',
 'B-LOC',
 'I-ORG',
 'B-FAC',
 'B-EVE',
 'B-TIM',
 'I-DAT',
 'B-MON',
 'B-PCT',
 'I-MON',
 'I-LOC',
 'I-FAC',
 'I-PRO',
 'I-TIM',
 'I-PER',
 'B-DAT',
 'B-ORG',
 'O',
 'B-PER']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [16]:
from pyspark.ml import Pipeline
from sparknlp.annotator import Tokenizer
from sparknlp.base import DocumentAssembler

document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    tokenClassifier_loaded
])

example = spark.createDataFrame([
    ["این سریال به صورت رسمی در تاریخ دهم می ۲۰۱۱ توسط شبکه فاکس برای پخش رزرو شد."],
    ['دفتر مرکزی شرکت کامیکو در شهر ساسکاتون ساسکاچوان قرار دارد.'],
    ['در سال ۲۰۱۳ درگذشت و آندرتیکر و کین برای او مراسم یادبود گرفتند.']
]).toDF("text")

result = pipeline.fit(example).transform(example)
result.select("text", "ner.result").show(truncate=False)

+----------------------------------------------------------------------------+--------------------------------------------------------------+
|text                                                                        |result                                                        |
+----------------------------------------------------------------------------+--------------------------------------------------------------+
|این سریال به صورت رسمی در تاریخ دهم می ۲۰۱۱ توسط شبکه فاکس برای پخش رزرو شد.|[O, O, O, O, O, O, O, O, O, O, O, B-ORG, I-ORG, O, O, O, O, O]|
|دفتر مرکزی شرکت کامیکو در شهر ساسکاتون ساسکاچوان قرار دارد.                 |[O, O, B-ORG, I-ORG, O, B-LOC, I-LOC, I-LOC, O, O, O]         |
|در سال ۲۰۱۳ درگذشت و آندرتیکر و کین برای او مراسم یادبود گرفتند.            |[O, B-DAT, I-DAT, O, O, B-LOC, O, B-PER, O, O, O, O, O, O]    |
+----------------------------------------------------------------------------+--------------------------------------------------------------+



That's it! You can now go wild and use hundreds of `AlbertForTokenClassification` models from HuggingFace 🤗 in Spark NLP 🚀
